In [1]:
import re
import json
import numpy as np
import pandas as pd
import jieba
from zhconv import convert
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA

In [ ]:
# 读取20220831的原始cv和jd特征数据
all_data = pd.read_csv('../data_20220831/raw_cvjd_20220831_spark.csv').drop(['Unnamed: 0'], axis=1)